In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import wave
import openpyxl
import yaml
import os
import shutil
import _pickle as cpl
import tarfile
import pandas as pd
%matplotlib inline

In [2]:
# Get all the recordings and separate them by format. 
flac_files = []
wav_files = []
for file in os.listdir("../../dataset/recordings"):
    if file.endswith(".flac"):
        flac_files.append(file)
    elif file.endswith(".wav"):
        wav_files.append(file)

In [3]:
# Convert all the .flac files to .wav files and store them in 'wav_recordings'
os.makedirs("../../dataset/wav_recordings")
for i in range(len(flac_files)):
    string = 'sox ../../dataset/recordings/' + str(flac_files[i]) + ' ../../dataset/wav_recordings/' + str(flac_files[i][:-5]) + '.wav'
    os.system(string)

In [4]:
# Store the .wav files in 'wav_recordings'
for i in range(len(wav_files)):
    string = 'mv ../../dataset/' + str(wav_files[i]) + ' ../../dataset/wav_recordings'
    os.system(string)

In [6]:
# Loads the validation data
df = pd.ExcelFile('../../dataset/validationsAndROIs.xlsx')
df = df.parse('ROIs')
rs =[]


# Gets the name of all the rcordings
all_wav_files = []
for file in os.listdir("../../dataset/wav_recordings"):
    all_wav_files.append(file[:file.index('.')])

# Extracts the recording_name column and stores it as a list
recording_name = df["recording name"].tolist()

# Formats the string containing the name of the recording to remove everything after the first dot.
for i in range(len(recording_name)):
    recording_name[i] = recording_name[i][:recording_name[i].index('.')]

# If we don't have a recording, then the validation data is not useful by itself. So we want to remove this data. 
for i in range(len(recording_name)):
    if recording_name[i] not in all_wav_files:
        rs.append(recording_name[i])
        recording_name[i] = "delete"

# Creates the column 'recording name' initialized with the corresponding values in tecording_name list, 
# then removes the data labeled as delete

df['recording name'] = recording_name
df = df[df['recording name'] != 'delete']

# Writes out the corrected validation data. 
writer = pd.ExcelWriter('../../dataset/corrected_validationsAndROIs.xlsx')
df.to_excel(writer, 'ROIs', index=False)
writer.save()

# Standarizes the names of all the recording names
for files in os.listdir("../../dataset/wav_recordings"):
    new_name = files[:files.index('.')]
    os.rename("../../dataset/wav_recordings/" + files, "../../dataset/wav_recordings/" + new_name + '.wav')

In [8]:
grabaciones = list(df['recording name'])

In [9]:
grabaciones = [grabacion + '.wav' for grabacion in grabaciones]

In [10]:
# Get the Info from the .wav file.
def wavInfo(rec_file):
    wav_file = wave.open(rec_file, 'r')
    frames = wav_file.readframes(-1)
    wave_info = pylab.fromstring(frames, 'Int16') #all .wavs in our dataset are 16bit
    framerate = wav_file.getframerate()
    wav_file.close()
    return wave_info, framerate

# Get the info from the Spectrogram, but don't plot it.
def specInfo(rec_file_list):
    for rec_file in rec_file_list:
        new_file = '../../dataset/spectrograms/' + rec_file + '.png'
        rec_file = '../../dataset/wav_recordings/'+rec_file
        wave_info, framerate = wavInfo(rec_file)
        spectrum, freqs, t, _ = pylab.specgram(wave_info, 
                                               NFFT=512, 
                                               noverlap=256, 
                                               window=pylab.window_hanning, 
                                               Fs=framerate)
        pylab.axis('off')
        pylab.savefig(new_file,
                      bbox_inches='tight', transparent = True)
        pylab.close()
        
        

In [13]:
os.makedirs("../../dataset/spectrograms")
spectrum = specInfo(grabaciones)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


In [14]:
del spectrum

In [21]:
# Creates the speciesData dictionary which is a dict with all the information we have regarding the different species. 
def speciesData(workbook):
    roi_ws = openpyxl.load_workbook(workbook)['ROIs']
    dataset = {}
    # needed format:
    # species specimen per row
    # columns: species name, start_time, end_time, min_freq, max_freq, recording name
    # columns A to F
    sheetMatrix = list(roi_ws.iter_rows())
    # remove row with column names and create array of keys per species. (e.g. start_time, end_time, ...)
    keys = sheetMatrix.pop(0) 
    for row in sheetMatrix:
        speciesName = row[0].value
        if (speciesName not in dataset):
            dataset[speciesName] = {}
        for col in range(1,len(row)):
            cell = ''
            # change recording extension since we are dealing with wav files
            if (col == 5):
                cell = row[col].value
                cell += '.wav'
            else:
                cell = row[col].value
            # if per species key is not present add the key and add the value as the first element in a list
            if (keys[col].value not in dataset[speciesName]): 
                dataset[speciesName][keys[col].value] = [cell]
            # append to the list of attributes 
            else:
                dataset[speciesName][keys[col].value] = dataset[speciesName][keys[col].value] + [cell]
    return dataset

# Convert speciesData dictionary to yaml and save file
def dataToYAML(data, name): 
    # need to check if file exists then delete it
    path = name
    dataset = open(path, 'w+')
    dump = yaml.dump(data, dataset, default_flow_style=False)
    dataset.close()

# As it names suggests, it find the maximum. 
def findMax(L):
    Max = float('-inf')
    for n in L:
        if (n > Max):
            Max = n
    return Max

# As it names suggests, it find the minimum. 
def findMin(L):
    Min = float('inf')
    for n in L:
        if (n < Min):
            Min = n
    return Min

In [22]:
# save our species data dictionary as a .yaml file for later use
workbook = '../../dataset/corrected_validationsAndROIs.xlsx'
data = speciesData(workbook)
dataToYAML(data, '../../dataset/dataset.yaml')

In [23]:
yamlData = open('../../dataset/dataset.yaml', 'r')
dataset = yaml.load(yamlData)
yamlData.close()

In [24]:
# search for the index of the leftmost value in an ordered array 
# (of times or frequencies in our case) that still meet our criteria
def leftmostBinSearch(A, lo, hi, target):
    mid = (lo + hi) // 2
    v1 = A[mid]
    if (v1 >= target):
        if (mid > 0 and A[mid - 1] > target):
            return leftmostBinSearch(A, lo, mid-1, target)
        else:
            return mid
    elif (A[mid] < target):
        return leftmostBinSearch(A, mid+1, hi, target)
    else:
        return leftmostBinSearch(A, lo, mid-1, target)

# search for the index of the rightmost value in an ordered array 
# (of times or frequencies in our case) that still meet our criteria
def rightmostBinSearch(A, lo, hi, target): # something is wrong and it's giving me 1 to the right 
    mid = (lo + hi) // 2
    v1 = A[mid]
    if (v1 <= target):
        if (mid < (len(A) - 1) and A[mid + 1] <= target):
            return rightmostBinSearch(A, mid+1, hi, target)
        else:
            return mid
    elif (A[mid] < target):
        return rightmostBinSearch(A, mid+1, hi, target)
    else:
        return rightmostBinSearch(A, lo, mid-1, target)
    
# Calls on rightmostBinSearch and leftmostBinSearch
def getBounds(A, minVal, maxVal):
    left = leftmostBinSearch(A, 0, len(A)-1, minVal)
    right = rightmostBinSearch(A, 0, len(A)-1, maxVal)
    return left, right

In [25]:
y = []
x = []

In [26]:
def getRawSpecDataset(dataset, path='../../dataset'):
    
    # make directory to store our spec dataset
    dataset_path = path + '/spectrogram_roi_dataset'
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    else:
        shutil.rmtree(dataset_path)
        os.makedirs(dataset_path)
    species = dataset.keys()
    
    # image data to be pickled 
    specs = []
    
    for s in species:
        #s_dir = dataset_path + '/' + s
        s_spec = []
        #os.makedirs(s_dir) # make a directory per species
        
        # load species ROI data
        min_freqs = dataset[s]['min_frequency']
        max_freqs = dataset[s]['max_frequency']
        starts = dataset[s]['start_time']
        ends = dataset[s]['end_time']
        recs = dataset[s]['recording name']
        print(s)
        
        for i in range(0, len(recs)):
            
            rec = '../../dataset/wav_recordings/' + recs[i] # path to ith recording file where s is present
            
            wave_info, framerate = wavInfo(rec)
            spectrum, freqs, times, _ = pylab.specgram(wave_info, 
                                               NFFT=512, 
                                               noverlap=256, 
                                               window=pylab.window_hanning, 
                                               Fs=framerate)
            freqs = np.asarray(freqs)
            #freqs = freqs.flatten(freqs)
            
            x.append(freqs)
            pylab.axis('off')
            pylab.close()
            
            #spectrum, freqs, times = specInfo(rec) # get entire spectrogram data from rec
            
            # get ROI info in rec
            t_0 = starts[i] 
            t_n = ends[i]
            f_0 = min_freqs[i]
            f_n = max_freqs[i]
            
            
            # find closest times and freqs that match ROI info
            t_start, t_end = getBounds(times, t_0, t_n)
            f_start, f_end = getBounds(freqs, f_0, f_n)
            
            '''
            # get modified spectrum, freqs, and times
            #spectrumMod = specMod(spectrum, freqs, times, f_start, f_end, t_start, t_end)
            freqMod = freqs[f_start:f_end]
            timeMod = times[t_start:t_end]
            
            '''
            y.append([t_0, f_0, t_n, f_n])
            
            #np.array([t_start, f_start, t_end, f_end])
            
    #return np.array([t_0, f_0, t_n, f_n])

In [27]:
data = getRawSpecDataset(dataset)

Basileuterus bivittatus


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


Basileuterus chrysogaster
Chlorothraupis carmioli
Eleutherodactylus brittoni
Eleutherodactylus cochranae
Eleutherodactylus cooki
Eleutherodactylus coqui
Eleutherodactylus juanariveroi
Epinephelus guttatus
Formicarius analis
Hypocnemis subflava
Liosceles thoracicus
Megascops guatemalae
Megascops nudipes
Microcerculus marginatus
Myrmeciza hemimelaena
Myrmoborus leucophrys
Percnostola lophotes
Saltator grossus
Thamnophilus schistaceus
Unknown Insect


In [28]:
y[0]

[40.7895215572, 1805.81896552, 43.1253410951, 5341.42241379]

In [29]:
x[0]

array([    0.       ,    86.1328125,   172.265625 ,   258.3984375,
         344.53125  ,   430.6640625,   516.796875 ,   602.9296875,
         689.0625   ,   775.1953125,   861.328125 ,   947.4609375,
        1033.59375  ,  1119.7265625,  1205.859375 ,  1291.9921875,
        1378.125    ,  1464.2578125,  1550.390625 ,  1636.5234375,
        1722.65625  ,  1808.7890625,  1894.921875 ,  1981.0546875,
        2067.1875   ,  2153.3203125,  2239.453125 ,  2325.5859375,
        2411.71875  ,  2497.8515625,  2583.984375 ,  2670.1171875,
        2756.25     ,  2842.3828125,  2928.515625 ,  3014.6484375,
        3100.78125  ,  3186.9140625,  3273.046875 ,  3359.1796875,
        3445.3125   ,  3531.4453125,  3617.578125 ,  3703.7109375,
        3789.84375  ,  3875.9765625,  3962.109375 ,  4048.2421875,
        4134.375    ,  4220.5078125,  4306.640625 ,  4392.7734375,
        4478.90625  ,  4565.0390625,  4651.171875 ,  4737.3046875,
        4823.4375   ,  4909.5703125,  4995.703125 ,  5081.8359

In [30]:
images = np.asarray(x)
bbox = np.asarray(y)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, bbox, test_size=0.33, random_state=42)

#X_train = np.asarray(X_train)
#y_train = np.asarray(y_train)

#X_test = np.asarray(X_test)
#y_test = np.asarray(y_test)

In [37]:
# Build the model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
model = Sequential([
        Dense(4096, input_dim=X_train.shape[-1]),
        Dense(4096),
        Dense(4096),
        Dense(4096),
        Dense(4096),
        Activation('relu'), 
        Dropout(0.2), 
        Dense(y_train.shape[-1])
    ])
model.compile('adadelta', 'mse')

In [ ]:
model.fit(X_train, y_train, nb_epoch=30, validation_data=(X_test, y_test), verbose=2)

/usr/local/lib64/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 213 samples, validate on 106 samples
Epoch 1/30
